In [1]:
import matplotlib.pyplot as plt
#from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [2]:
import pickle

In [3]:
full_df = pd.read_csv("../../Data/creditcard.csv")
full_df.drop(columns='Time', inplace=True)

In [4]:
fraud_df = full_df.loc[full_df['Class'] == 1]
fraud_df = fraud_df.reset_index(drop=True)

In [5]:
not_fraud_df = full_df.loc[full_df['Class']==0]
not_fraud_df = not_fraud_df.sample(n=492, random_state=1)
not_fraud_df = not_fraud_df.reset_index(drop=True)

In [6]:
new_df = fraud_df.append(not_fraud_df, ignore_index=True)

In [7]:
new_df.shape

(984, 30)

In [8]:
target = new_df["Class"]
target_names = ["negative", "positive"]

In [9]:
data = new_df.drop("Class", axis=1)
feature_names = data.columns
data.head()

V1        V2        V3        V4        V5        V6        V7  \
0 -2.312227  1.951992 -1.609851  3.997906 -0.522188 -1.426545 -2.537387   
1 -3.043541 -3.157307  1.088463  2.288644  1.359805 -1.064823  0.325574   
2 -2.303350  1.759247 -0.359745  2.330243 -0.821628 -0.075788  0.562320   
3 -4.397974  1.358367 -2.592844  2.679787 -1.128131 -1.706536 -3.496197   
4  1.234235  3.019740 -4.304597  4.732795  3.624201 -1.357746  1.713445   

         V8        V9       V10  ...       V20       V21       V22       V23  \
0  1.391657 -2.770089 -2.772272  ...  0.126911  0.517232 -0.035049 -0.465211   
1 -0.067794 -0.270953 -0.838587  ...  2.102339  0.661696  0.435477  1.375966   
2 -0.399147 -0.238253 -1.525412  ... -0.430022 -0.294166 -0.932391  0.172726   
3 -0.248778 -0.247768 -4.801637  ... -0.171608  0.573574  0.176968 -0.436207   
4 -0.496358 -1.282858 -2.447469  ...  0.009061 -0.379068 -0.704181 -0.656805   

        V24       V25       V26       V27       V28  Amount  
0  0.320198  0.044519  0.177840  0.261145 -0.143276    0.00  
1 -0.293803  0.279798 -0.145362 -0.252773  0.035764  529.00  
2 -0.087330 -0.156114 -0.542628  0.039566 -0.153029  239.93  
3 -0.053502  0.252405 -0.657488 -0.827136  0.849573   59.00  
4 -1.632653  1.488901  0.566797 -0.010016  0.146793    1.00  

[5 rows x 29 columns]

In [10]:
target.shape

(984,)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [12]:
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [17]:
from sklearn.svm import SVC

model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [22]:
filename = 'fraud_model2scaled.sav'
#model = pickle.load(open(filename, 'rb'))

In [19]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test_scaled, y_test))

Test Acc: 0.919


In [20]:
# Calculate classification report
from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

    negative       0.86      1.00      0.92       120
    positive       1.00      0.84      0.91       126

    accuracy                           0.92       246
   macro avg       0.93      0.92      0.92       246
weighted avg       0.93      0.92      0.92       246



In [23]:
pickle.dump(model, open(filename, 'wb'))